In [59]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import TransformedTargetRegressor
from sklearn.covariance import GraphicalLassoCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_validate, GridSearchCV, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer

In [9]:
expr = pd.read_csv('../data/albert2018/processed/albert2018_expression_logtpm.tsv', sep='\t', index_col=0)

In [10]:
expr = expr.T

In [11]:
cov = pd.read_csv('../data/albert2018/processed/albert2018_segregant_covariates.tsv', sep='\t', index_col=0)

In [12]:
gen = pd.read_csv('../data/albert2018/processed/albert2018_genotypes.tsv', sep='\t', index_col=0)

In [13]:
gen_meta = gen.loc[:, ['chr', 'pos']]

gen = (
    gen
    .drop(columns=['chr', 'pos', 'ref', 'alt'])
    .transpose()
    .apply(lambda x: (x + 1) / 2)
    .astype(np.int8)
)

In [14]:
annot = pd.read_csv('../data/albert2018/processed/albert2018_genes.tsv', sep='\t', index_col=0)

In [15]:
cis_window = 20_000

In [16]:
expr = expr.loc[:, expr.columns.isin(annot.index)]

In [17]:
gene_Xy = dict()

for gene in expr.columns:
    gene_y = expr[gene]
    
    gene_chr, gene_tss = annot.loc[gene, ['chromosome_name', 'transcription_start_site']]
    
    gene_X = gen.loc[
        :, (gen_meta['chr'] == gene_chr) & (gen_meta['pos'].between(gene_tss - cis_window, gene_tss + cis_window))
    ]

    gene_Xy[gene] = [gene_X, gene_y]

In [46]:
hparam_grid = {
    'ttr__regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'ttr__regressor__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9, 1]
}

In [52]:
enet = Pipeline([(
    'ttr', TransformedTargetRegressor(
        regressor=ElasticNet(tol=1e-4), 
        transformer=QuantileTransformer(n_quantiles=100, output_distribution='normal')
    )
)])

In [53]:
inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=51)

In [54]:
clf = GridSearchCV(estimator=enet, param_grid=hparam_grid, cv=inner_cv)

In [62]:
nested_score = cross_validate(clf, X=gene_Xy['YAL062W'][0], y=gene_Xy['YAL062W'][1], cv=outer_cv, n_jobs=-1, return_estimator=True)

In [79]:
nested_score['test_score'].argmax()

2

In [80]:
nested_score['estimator'][nested_score['test_score'].argmax()].best_params_

{'ttr__regressor__alpha': 0.1, 'ttr__regressor__l1_ratio': 0.3}

In [102]:
genet = make_pipeline(
    TransformedTargetRegressor(
        regressor=ElasticNetCV(
            l1_ratio=[.1, .5, .7, .9, .95], 
            tol=1e-2, 
            cv=ShuffleSplit(n_splits=5, random_state=42)
        ),
        transformer=QuantileTransformer()
    )
)

In [142]:
elastic_net_cv = gene_models['YAL061W'].named_steps['transformedtargetregressor'].regressor_

In [143]:
list(elastic_net_cv.alphas_).index(elastic_net_cv.alpha_)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [126]:
# Get the mean squared errors for each fold and alpha
mse_path = elastic_net_cv.mse_path_  # Shape: (n_alphas, n_folds)

# Get the optimal alpha index
optimal_alpha_index = list(elastic_net_cv.alphas_).index(elastic_net_cv.alpha_)

# Mean squared error for the optimal alpha across folds
mse_per_fold = mse_path[optimal_alpha_index]

# Calculate R^2 for each fold
fold_r2_scores = [1 - mse / np.var(y_train) for mse in mse_per_fold]
print("R^2 Scores for Test Folds during Cross-Validation:", fold_r2_scores)

# Mean R^2 score across folds
mean_r2_cv = np.mean(fold_r2_scores)
print("Mean R^2 Score from Cross-Validation:", mean_r2_cv)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [116]:
cross_val_score(genet, gene_Xy['YAL062W'][0], gene_Xy['YAL062W'][1], cv=ShuffleSplit(n_splits=5, random_state=42), n_jobs=-1)

array([0.34214196, 0.40720485, 0.28155676, 0.36784505, 0.25818627])